## BLUF:
The following workflow describes how to perform the following tasksed used in a recent Twitter field experiment


## 1. Identify users who share article links from untrustworthy websites

In [1]:
import pandas as pd
import configparser

# Setting config values
config = configparser.ConfigParser()
config.read("/Users/dankoban/Documents/Dissertation/config.ini")
CONSUMER_KEY = config['twitter']['consumer_key_bot']
CONSUMER_SECRET = config['twitter']['consumer_secret_bot']
ACCESS_KEY = config['twitter']['access_key_bot']
ACCESS_SECRET = config['twitter']['access_secret_bot']
BEARER_TOKEN = str(config['twitter']['bearer_token_bot'])

Connect to endpoint

In [2]:
import tweepy
import json
import os
import pandas as pd
import time

Client = tweepy.Client(bearer_token=BEARER_TOKEN, 
                       consumer_key=CONSUMER_KEY, 
                       consumer_secret=CONSUMER_SECRET, 
                       access_token=ACCESS_KEY, 
                       access_token_secret=ACCESS_SECRET, 
                       wait_on_rate_limit=True)

Search for a specific article that was fact checked to be false

In [ ]:
run = False
if run == True:
    r = Client.search_all_tweets(query="(ukraine-donated-the-most-cash-of-any-country-to-the-clinton-foundation) (lang:en) -is:retweet",                                               
                                 start_time='2019-01-01T00:00:00.00Z',  
                                 end_time='2021-06-01T00:00:00.00Z', 
                                 max_results=500,                                                  
                                 tweet_fields=['author_id',#'context_annotations',
                                               'conversation_id',
                                               'created_at','entities','geo','id',
                                               'in_reply_to_user_id','lang',                                          
                                               'public_metrics','referenced_tweets',
                                               'reply_settings','source','text','withheld']) 
    for i in range(0, len(r.data)):
        status_id = r.data[i].id                 
        r_dump = json.dumps(r.data[i].data, indent = 4)                               
        with open('/Users/dankoban/Documents/fe_workflow_data/primary_tweets/'+ str(status_id) + '.json', 'w', encoding='utf-8') as f:
                   f.write(r_dump)  

In [3]:
files = os.listdir('/Users/dankoban/Documents/fe_workflow_data/primary_tweets/')
files = [file for file in files if '.json' in file]

primary_tweets = []
for file in files:
    # JSON file    
    f = open ('/Users/dankoban/Documents/fe_workflow_data/primary_tweets/' + file, "r")
  
    # Reading from file
    data = json.loads(f.read())    
    tweet_df = pd.json_normalize(data)
    try:
        tweet_df['in_reply_to_user_id'] = tweet_df['in_reply_to_user_id'] 
    except:
        tweet_df['in_reply_to_user_id'] = None
    try:
        tweet_df['referenced_tweets_type'] = data['referenced_tweets'][0]['type']
        tweet_df['referenced_tweets_id'] = data['referenced_tweets'][0]['id']
    except:
        tweet_df['referenced_tweets_type'] = None
        tweet_df['referenced_tweets_id'] = None
    try:
        tweet_df['expanded_url'] = data['entities']['urls'][0]['expanded_url']
    except:
        tweet_df['expanded_url'] = None
    try: 
        tweet_df['source'] = data['source']
    except:
        tweet_df['source'] = None
    tweet_df = tweet_df[['author_id','id','conversation_id','text','lang',
                         'public_metrics.retweet_count',
                         'public_metrics.reply_count',
                         'public_metrics.like_count',
                         'public_metrics.quote_count',
                         'referenced_tweets_type','referenced_tweets_id',
                         'in_reply_to_user_id', 'expanded_url', 'source']]
    primary_tweets.append(tweet_df)

primary_tweets = pd.concat(primary_tweets).reset_index(drop=True)
#primary_tweets = primary_tweets[primary_tweets['expanded_url'].isin(['https://www.lifezette.com/2019/10/ukrainians-cash-clinton-foundation/',
#                                                                      'https://www.lifezette.com/2019/10/ukraine-cash-clinton-foundation/',
#                                                                      'https://www.lifezette.com/2019/10/ukrainians-cash-clinton-foundation/#gf_2'])]
primary_tweets.reset_index(inplace = True, drop = True)
primary_tweets['expanded_url'].value_counts().head(3)

https://truepundit.com/ukraine-donated-the-most-cash-of-any-country-to-the-clinton-foundation/    86
https://www.lifezette.com/2019/10/ukraine-cash-clinton-foundation/                                60
https://www.waynedupree.com/clinton-foundation-ukraine/                                           13
Name: expanded_url, dtype: int64

Identify who shared these articles

In [4]:
authors = primary_tweets['author_id'].unique().tolist()
status_ids = primary_tweets['id'].unique().tolist()
print(str(len(status_ids)) + " tweets were posted by " + str(len(authors)) + " users")

201 tweets were posted by 175 users


## 2. Programmatically follow accounts to generate followers via follow-backs

In [5]:
Client.get_users_following(id = '1397202218156167168')

Response(data=[<User id=3039220281 name=Dan Koban username=kobo6992>], includes={}, errors=[], meta={'result_count': 1})

In [6]:
Client.follow('3039220281')
Client.get_users_following(id = '1397202218156167168')

Response(data=[<User id=3039220281 name=Dan Koban username=kobo6992>], includes={}, errors=[], meta={'result_count': 1})

In [ ]:
#Client.unfollow('3039220281')
#Client.get_users_following(id = '1397202218156167168')

## 3. Collect social network and profile information about individual users

In [ ]:
#Client.get_users_following('3039220281')
#Client.get_users_tweets('3039220281')

## 4. Programmatically message users to rate the accuracy of a headline

In [7]:
# authorization of consumer key and consumer secret
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)

# set access to user's access key and access secret 
auth.set_access_token(ACCESS_KEY, ACCESS_SECRET)
  
# calling the api 
api = tweepy.API(auth)
  
# ID of the recipient
recipient_id = '3039220281'
  
# text to be sent
text = "Hi, I'm a bot.  Can you rate the accuracy of this headline? Thanks : )"
  
# sending the direct message
direct_message = api.send_direct_message(recipient_id, text)

## 5. Measure changes to posting activity of individual users

In [ ]:
#Client.get_users_following('3039220281')
#Client.get_users_tweets('3039220281')